In [168]:
# Library
import numpy as np
import pandas as pd
import string
from datetime import datetime
from pathlib import Path

In [169]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def wordprocess(s):
    #s=str(s)
    s = str(s).lower()
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', s)
    tokenizer = RegexpTokenizer(r'p&l|m&a|[\u4e00-\ufaff]+|[0-9]+|[a-zA-Z]+\'*[a-z]*')
    #tokenizer = RegexpTokenizer(r"\s|[\.,;'/\\\-:\(\)_]", gaps=True)
    tokens = tokenizer.tokenize(cleantext)  
    #print(tokens)
    return tokens

In [190]:
Data.iloc[0].Suffix_File.strip() in File_Suffix.Suffix.values


True

In [193]:
'docx' in File_Suffix.Suffix.values

False

### Handle invalid suffix

In [197]:
Data = pd.read_csv('v_usb_usage_report_20220825.csv', encoding='utf-8')
File_Suffix= pd.read_csv('File_Suffix.csv')
Data_filename = Data['File_Name']
Data['File_size'] = [x.rpartition('-')[2] for x in Data['File_Name']]
Data['Suffix_File'] = [x.rpartition('-')[0].rpartition('.')[2] for x in Data['File_Name']]
Data['file_suffix'] = Data.apply(lambda x: x.Suffix_File if x.Suffix_File.strip().lower() in File_Suffix.Suffix.values else 'None', axis=1)
Data['word_list']=Data['File_Name'].map(lambda s:wordprocess(s))
del Data['Suffix_File']
# Data[Data['file_suffix'] == 'None']

In [170]:
# # %%Data
# Data = pd.read_csv('v_usb_usage_report_20220825.csv', encoding='utf-8')
# Data_filename = Data['File_Name']
# Data['File_size'] = [x.rpartition('-')[2] for x in Data['File_Name']]
# Data['File_suffix'] = [x.rpartition('-')[0].rpartition('.')[2] if len(x.rpartition('-')[0].rpartition('.')[2])<8 else None for x in Data['File_Name']]
# Data['word_list']=Data['File_Name'].map(lambda s:wordprocess(s)) 
# # Data

In [171]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmayuxi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jmayuxi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jmayuxi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [5]:
pip install zhconv

Note: you may need to restart the kernel to use updated packages.


In [198]:
Data_Keywords = pd.read_csv('key_words_collection.csv')

In [199]:
from zhconv import convert
cn_keywords = [cn.strip() for cn in Data_Keywords['Chinese Keywords'] if not pd.isnull(cn)]
Keywords_lst = [eng.lower() for eng in Data_Keywords['English Keywords']  if not pd.isnull(eng)] +\
[convert(cn, 'zh-tw') for cn in cn_keywords] + cn_keywords
# Keywords_lst

In [174]:
# def SensitiveWordsMatch(word_list, Keywords_lst):
#     # if word in file name is English then we do exact match, if it is cn char then we can just fit in.
#     output = []
#     for word in word_list:
#         # if word is an English word
#         if (word.isascii()) and (word in Keywords_lst):
#             output.append(word)
# #       if word is in other language
#         elif (not word.isascii()):
# #             output.append(word if any(word in key for key in Keywords_lst))
#             for key in Keywords_lst:
#                 if key in word:
#                     output.append(key)
#     return output
            

### If a word is an English word, we apply exact match in key word list; otherwise substring is acceptable.

In [200]:
def SensitiveWordsMatch(word_list, Keywords_lst):   
    output = []
    file_name_merge = ''.join(word_list)
    for word in word_list:
        if word.isascii():
            for key in Keywords_lst:
                # if key.isspace() == True:
                if ' ' in key:
                    key_split = key.split(' ', -1)
                    key_merge = ''.join(key_split)
                    key_reverse = key_split[1] + key_split[0]
                    if key_merge.lower() in file_name_merge or key_reverse.lower() in file_name_merge:
                        output.append(key)
                elif key == word:
                    output.append(key)
        else:
            for key in Keywords_lst:
                if key in word:
                    output.append(key)
    return(list(set(output)))

In [176]:
# key = ['open blue'] # 拆解成 key = ['open blue', 'open', 'blue']
# word = 'openblue' # 被识别
# word = 'open', 'otherwords','blue' # 不被识别？
# word = 'blueopen' # 被识别？


In [201]:
Data['sensitive_words'] = Data['word_list'].map(lambda s:SensitiveWordsMatch(s, Keywords_lst))
Data['sensitive_flag'] = [ 1 if len(x) != 0 else 0 for x in Data['sensitive_words']]
# Data

In [178]:
# del Data['word_list']

In [203]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
Data[Data['sensitive_flag']== 1][['File_Name','File_size','file_suffix','sensitive_words']]

,File_Name,File_size,file_suffix,sensitive_words
16,C:\我的文档\a-Product mamagement-Total Asia\BLC Meeting\Sep20&21 in Wuxi\副本JCH-GA Price comparison with WX MLO_0918.xlsx - 106.36 KB,106.36 KB,xlsx,[price]
74,C:\黎金1\广西报价\南宁华润\2018-4-14主板高压条配件\附件2：《关于万象城制冷中心空调主机主板采购项目询价文件》.docx - 51.51 KB,51.51 KB,docx,[报价]
75,C:\黎金1\广西报价\改造项目\改造项目信息收集指导7.4.pptx - 3.6 MB,3.6 MB,pptx,[报价]
76,C:\黎金1\广西报价\改造项目\改造业务培训7.11.pptx - 1.84 MB,1.84 MB,pptx,[报价]
84,C:\陈勇\方案书\泰州\海阳化纤\江边工厂\20180526\合同\与约克的合同\招标新格式\attachment\启动柜招标技术要求.docx - 28.11 KB,28.11 KB,docx,[合同]
85,C:\陈勇\方案书\泰州\海阳化纤\江边工厂\20180526\合同\与约克的合同\启动柜招标技术要求.docx - 28.11 KB,28.11 KB,docx,[合同]
86,C:\陈勇\方案书\泰州\海阳化纤\江边工厂\20180526\合同\与约克的合同\招标新格式\attachment\高压柜\启动柜招标技术要求.docx - 28.11 KB,28.11 KB,docx,[合同]
87,C:\陈勇\方案书\泰州\海阳化纤\总厂\20180905\合同\启动柜技术要求(招标用）.docx - 32.53 KB,32.53 KB,docx,[合同]
88,C:\陈勇\方案书\泰州\海阳化纤\江边工厂\20180526\合同\与约克的合同\招标新格式\attachment\高压柜\最终报批\江苏海阳5_、6_聚合项目高压离心式冷水机组ESG-高压控制柜外采等-上海艾昆185000.7z - 2.25 MB,2.25 MB,7z,[合同]
89,C:\陈勇\方案书\扬州\仪化东丽\合同\CAF(V14)- 仪征化纤东丽公司项目.xlsm - 196.75 KB,196.75 KB,xlsm,[合同]
